https://huggingface.co/collections/google/medgemma-release-680aade845f90bec6a3f60c4 \
https://huggingface.co/google/medgemma-4b-it

In [9]:
import os
import sys

In [10]:
# Set CUDA_VISIBLE_DEVICES to expose only device 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set CUDA_VISIBLE_DEVICES to expose devices 0, 1, 2, and 3
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [11]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import requests
import torch
import pandas as pd

#### Reading the Pathology Test Data

In [12]:
questions = pd.read_csv("../pathology_test_data/questions.csv")
questions

,image_name,question,original_responses
0,ck_PTGC_2x.jpeg,How can you best describe the low-power patter...,The lymph node shows a mixed follicular and in...
1,ck_PTGC_2x.jpeg,What are three main differential diagnostic co...,"Based on the low-power pattern, the primary co..."
2,ck_PTGC_5x.jpeg,What is the morphologic alteration being depic...,"The image shows a central enlarged, somewhat i..."
3,ck_PTGC_5x.jpeg,What is the expected immunoarchitecture of the...,This image shows an enlarged secondary follicl...
4,ck_serositis_4x.jpg,What is the most common source for the change ...,"The changes here show extensive serositis, wit..."
5,ck_serositis_4x.jpg,What is the specific anatomic region shown in ...,The right half shows the muscularis propria an...
6,ck_steatohepatitis_100x.jpg,If these histologic changes included conspicuo...,There is fatty liver disease and if concurrent...
7,ck_steatohepatitis_100x.jpg,What would the primary histologic feature to s...,Wilson's disease has many non-specific finding...
8,ck_steatohepatitis_200x.jpg,In an overweight adolescent with mildly increa...,"In the background of steatosis, there is incre..."


In [13]:
images_path = os.path.join(os.getcwd(), "../pathology_test_data/images")
images_list = os.listdir(images_path)
images_list

['ck_steatohepatitis_100x.jpg',
 'ck_PTGC_2x.jpeg',
 'ck_serositis_4x.jpg',
 'ck_steatohepatitis_200x.jpg',
 'ck_PTGC_5x.jpeg']

#### Loading the Model Checkpoint and Image Processor

In [14]:
model_id = "google/medgemma-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]


In [15]:
processor = AutoProcessor.from_pretrained(model_id, padding=True, return_tensors="pt")

#### Creating the message structure

In [16]:
messages = []
for index, row in questions.iterrows():
    image_path = os.path.join(images_path, row["image_name"])
    question = row["question"]
    
    message = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."}]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": question}
            ]
        }
    ]   
    
    messages.append(message)

#### Running the batch inference

In [17]:
inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=True,
                                        return_dict=True, return_tensors="pt")

inputs = inputs.to(model.device)

In [18]:
with torch.inference_mode():
    generated_ids = model.generate(**inputs, max_new_tokens=1024, do_sample=False)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

In [19]:
output_texts = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [20]:
output_texts

["Based on the low-power image, the pattern appears to be a **lymph node**.\n\nHere's a more detailed description:\n\n*   **Overall Appearance:** The image shows a relatively uniform, solid tissue sample.\n*   **Cellularity:** The tissue is densely populated with cells, suggesting a high cellularity.\n*   **Lymphoid Follicles:** The presence of lymphoid follicles is a key characteristic of lymph nodes. These follicles are clusters of lymphocytes (a type of white blood cell) that are involved in the immune response.\n*   **Other Structures:** The image also shows other structures, such as blood vessels and connective tissue, which are part of the lymph node's architecture.\n\nIn summary, the image shows a lymph node with a high density of lymphocytes within lymphoid follicles, surrounded by blood vessels and connective tissue.\n",
 "Based on the image, the lymph node appears to be a normal lymph node. However, given the context of a young teenager with solitary adenopathy, here are thre

#### Storing the output Responses

In [21]:
os.makedirs("./responses", exist_ok=True)
questions['medgemma-4b-it-response'] = output_texts
questions.to_csv("./responses/medgemma_4b_it_responses.csv", index=False)